In [ ]:
import pandas as pd
import numpy as np

In [ ]:
file_path = "../data/25년_수확량 통계.csv"
# xls = pd.ExcelFile(file_path)
# sheet_names = xls.sheet_names
# print(sheet_names)

df= pd.read_csv(file_path)


print("--- [시작] 데이터 전처리(공백 및 타입 정리)를 시작합니다. ---")

# 2. [문자형] 컬럼 공백 제거 (앞/뒤 공백)
# 분석에 주로 사용되는 범주형 컬럼들
categorical_cols = [
    '지역', '코드번호', '경작자', '구분', 'CASE', '시비 처리', '작물', '품종'
]

for col in categorical_cols:
    if col in df.columns:
        df[col] = df[col].astype(str).str.strip()

# 2-1. [문자형] '품종' 컬럼의 *중간* 공백도 모두 제거
if '품종' in df.columns:
    df['품종'] = df['품종'].str.replace(r'\s+', '', regex=True)

print("범주형(문자) 컬럼 공백 정리 완료.")

# 3. [숫자형] 문자열로 저장된 숫자 컬럼 정리
# (쉼표, 공백이 포함된 컬럼들)
numeric_object_cols = [
    '면적(m2)', '면적(평)', '수확량(kg)', '건중량(kg)',
    '평당 수확량', '건조 수확량(kg/10a)', '총 질소 살포량(kg/10a)'
]

for col in numeric_object_cols:
    if col in df.columns:
        # 1. 문자로 변환 (결측치는 'nan' 문자열이 됨)
        s = df[col].astype(str)
        # 2. 쉼표(,) 제거
        s = s.str.replace(',', '', regex=False)
        # 3. 모든 공백(앞/뒤/중간) 제거
        s = s.str.replace(r'\s+', '', regex=True)

        # 4. 숫자로 변환 (변환 안되는 값은 NaN 처리)
        df[col] = pd.to_numeric(s, errors='coerce')

print("숫자형(문자) 컬럼 숫자 타입 변환 완료.")

print("--- [완료] 데이터 전처리 완료. ---")

# 4. [확인] 전처리 후 데이터 정보 출력
print("\n--- [처리]")

print(df.head())